In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Load data

In [46]:
PATH = '../data/heart.csv'
df = pd.read_csv(PATH)

In [47]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [48]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


# Model

In [49]:
categorial_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
target_feature = 'target'

In [50]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [51]:
def build_numerical_pipeline():
    numerical_pipeline = Pipeline(
        [
            ('scaler', StandardScaler()),
        ]
    )
    return numerical_pipeline

In [52]:
def build_categorial_pipeline():
    categorial_pipeline = Pipeline(
        [
            ('ohe', OneHotEncoder()),
        ]
    )
    return categorial_pipeline

In [53]:
def build_transformer(categorial_features, numerical_features):
    transformer = ColumnTransformer(
        [
            (
                'categorial_pipeline',
                build_categorial_pipeline(),
                categorial_features
            ),
            (
                'numerical_pipeline',
                build_numerical_pipeline(),
                numerical_features
            )
        ]
    )
    return transformer

In [54]:
def make_features(transformer, df, target_feature):
    target = df[target_feature]
    features = transformer.transform(df)
    return features, target

In [55]:
transformer = build_transformer(categorial_features, numerical_features)

In [56]:
transformer.fit(train_data)

ColumnTransformer(transformers=[('categorial_pipeline',
                                 Pipeline(steps=[('ohe', OneHotEncoder())]),
                                 ['sex', 'cp', 'fbs', 'restecg', 'exang',
                                  'slope', 'ca', 'thal']),
                                ('numerical_pipeline',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['age', 'trestbps', 'chol', 'thalach',
                                  'oldpeak'])])

In [57]:
train_features, train_target = make_features(
    transformer, train_data, target_feature
)

In [58]:
def train_model(features, target, model_type):
    if model_type == 'LogisticRegression':
        model = LogisticRegression()
    elif model_type == 'RandomForestClassifier':
        model = RandomForestClassifier(n_estimators=100, criterion='entropy')
    model.fit(features, target)
    return model

In [59]:
def evaluate_model(preds, target):
    return {
        'accuracy': accuracy_score(target, preds),
        'roc_auc': roc_auc_score(target, preds)
    }

In [60]:
log_reg = train_model(train_features, train_target, 'LogisticRegression')
rand_fcl = train_model(train_features, train_target, 'RandomForestClassifier')

In [61]:
def make_predictions(model, features):
    return model.predict(features)

In [62]:
test_features, test_target = make_features(
    transformer, test_data, target_feature
)

In [63]:
preds_logreg = make_predictions(log_reg, test_features)
preds_fcl = make_predictions(rand_fcl, test_features)

In [64]:
metrics_logreg = evaluate_model(
        preds_logreg,
        test_target
    )

metrics_fcl = evaluate_model(
        preds_fcl,
        test_target
    )

In [65]:
metrics_logreg

{'accuracy': 0.8852459016393442, 'roc_auc': 0.8857758620689655}

In [66]:
metrics_fcl

{'accuracy': 0.819672131147541, 'roc_auc': 0.8200431034482759}